In [ ]:
# IMPORTS
import PyWGCNA
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [ ]:
## Move out of the notebook folder to access datasets
working_dir = os.getcwd()
working_dir = working_dir.strip('notebooks')


## Load the dataset
geneExpression = working_dir + 'data/5xFAD_paper/expressionList.csv'
dataset = pd.read_csv(geneExpression)

## Make a subset to save RAM
subset_dataset_size = 1000
dataset = dataset.iloc[:, :subset_dataset_size] 
susbsetFile = working_dir + 'data/5xFAD_paper/expressionListSubset.csv' 
dataset.to_csv(susbsetFile, sep=',', index=False, encoding='utf-8') 

## Load the subset
geneExpressionsubset = working_dir + 'data/5xFAD_paper/expressionListSubset.csv'


In [ ]:
### LOADING REAL UNPUBLISHED DATA    -     NO PUSHING FOR THE RESULTS

## Move out of the notebook folder to access datasets
working_dir = os.getcwd()
working_dir = working_dir.strip('notebooks')

# Prepare all the files for loading
data_dir = working_dir + 'data/PROTECTED_DATA/BGI_Expression_Data/'
mrna_count = data_dir + 'CRC.SW.mRNA.symbol.count.csv'
sample_info_selection = data_dir + 'Sample_Info_Selection.csv'

IMPORTANT!!!!!!!!!!!!!!1
There is 20 more samples in the dataset than in the excel. This is caused by pacients where two samples were taken (healthy and cancer tissue), that in the excel are displayed as one single sample. 

As far as i understand, this pacients got two samples taken from the colon, but since the Transcriptomics data is from blood plasma, this 120 extra samples are not relevant for our analysis in Transcriptomics, since they would be the same samples.

In [ ]:
figures_dir = working_dir + 'results/WGCNA_'  # weird folder formatting, needs to be like this.

## Initialize the WGCNA with the dataset
pyWGCNA_model = PyWGCNA.WGCNA(name='5xFAD',
                              species='mus musculus',
                              geneExpPath=mrna_count,
                              outputPath=figures_dir,
                              save=True,
                              networkType="unsigned",
                              powers = None)


In [ ]:
pyWGCNA_model.preprocess(show = True)

In [ ]:
pyWGCNA_model.findModules()
# pyWGCNA_5xFAD.findModules(blockSize=1500, moreNetworkConcepts = True) # For some reason these don't work. Make the powerVector crash or block size goes into wrong functions

In [ ]:
pyWGCNA_model.updateSampleInfo(path=sample_info_selection, sep=',')

# add color for metadata
pyWGCNA_model.setMetadataColor('Gender', {'Female': 'green',
                                       'Male': 'yellow'})
pyWGCNA_model.setMetadataColor('Primary Site Disease', {'Rectum': 'darkviolet',
                                                        'Colon': 'deeppink'})
pyWGCNA_model.setMetadataColor('Histology Subtype', {'Adenocarcinoma': 'thistle',
                                                    'Mucinous Adenocarcinoma': 'plum'})
pyWGCNA_model.setMetadataColor('Tumour Stage', {'Stage IV': 'red',
                                          'Stage III': 'orange',
                                          'Stage II': 'yellow',
                                          'Stage I': 'green'})

In [ ]:
# Automatic dataset builder from biomart.
'''
geneList = PyWGCNA.getGeneList(dataset='mmusculus_gene_ensembl',
                               attributes=['ensembl_gene_id',
                                           'external_gene_name',
                                           'gene_biotype'],
                               maps=['gene_id', 'gene_name', 'gene_biotype'],
                               server_domain="http://ensembl.org/biomart")
'''

# We already have the file, so we just load it from the file
gene_list_path = working_dir + 'data/5xFAD_paper/geneList'
pyWGCNA_5xFAD.updateGeneInfo(path=gene_list_path, sep = ' ')


Some visualizations

In [ ]:
pyWGCNA_model.datExpr

#pyWGCNA_5xFAD.datExpr.obs.head(5)

In [ ]:
# Plotting the adjacency matrix as heatmap
plt.figure()
sns.heatmap(pyWGCNA_model.adjacency, cmap='hot')
plt.title("Heatmap of the adjacency matrix")
plt.savefig(working_dir + 'results/WGCNA_figures/' + "Heatmap of the adjacency matrix", dpi=1000)
plt.show()


In [ ]:
pyWGCNA_model.analyseWGCNA()

In [ ]:
pyWGCNA_model.top_n_hub_genes(moduleName="silver", n=50)
